In [93]:
!pip install pandas

!pip install seaborn
!pip install numpy

!pip install matplotlib

!pip install scikit-learn

In [4]:
import matplotlib.pyplot as plt

import matplotlib

import seaborn as sns

import numpy as np

import pandas as pd

from sklearn import neighbors, metrics

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder

from collections import Counter

import re

from sklearn.covariance import EllipticEnvelope

from sklearn.linear_model import LinearRegression

from sklearn.naive_bayes import GaussianNB

from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

# functions

In [4]:
def analyze_data(dataset: pd.core.frame.DataFrame, target_name: str,
                 max_percentage_diff: float): # TODO: use another approach of function realizing
    target_data_arr = dataset[target_name].values
    dataset = dataset.drop([target_name], axis=1)
    trash_features = []
    for feature_name in dataset.keys():
        feature_data = dataset[feature_name]
        feature_data_dict = dict() # feature_data_dict =
        # {<feature_i-data>:  {
        #                     "count": <numb of same feature_i-data>,
        #                     "pos_target": <numb of positive target for objects with this feature_i-data>,
        #                     "neg_target": <numb of negative target for objects with this feature_i-data>
        #                     }
        # }

        for idx, feat_i in enumerate(feature_data):
            if feature_data_dict.get(feat_i, None) is None:
                feature_data_dict[feat_i] = {"count": 1, "pos_target": 0, "neg_target": 0}
            else:
                feature_data_dict[feat_i]["count"] += 1

            if target_data_arr[idx] == 0:
                feature_data_dict[feat_i]["neg_target"] += 1
            else:
                feature_data_dict[feat_i]["pos_target"] += 1

        print(f"\n{feature_name} pos/neg target ratio:")
        feature_i_percentages = []
        for i in feature_data_dict.items():
            pos = i[1]['pos_target'] # number of positive target for objects with this feature_i-data
            neg = i[1]['neg_target'] # number of negative target for objects with this feature_i-data
            if neg == 0 or pos == 0:
                info_str = f"{feature_name} : {i[0]} ,ratio = {pos} / {neg}"
            else:
                info_str = f"{feature_name} : {i[0]} ,ratio = {pos} / {neg} = {round(pos / neg, 3)} " \
                           f"\tpos_perc = {round(100 * pos / (pos + neg), 2)}%, " \
                           f"neg_perc = {round(100 * neg / (pos + neg), 2)}%"
                feature_i_percentages.append(round(100 * pos / (pos + neg), 2))
            print(info_str)
        if (max(feature_i_percentages) - min(feature_i_percentages)) < max_percentage_diff:
            trash_features.append(feature_name)
    return trash_features


NameError: name 'pd' is not defined

In [5]:
preproc_data = pd.read_csv("pre-processing_BankChurners.csv")
preproc_data_x = preproc_data.drop(["Attrition_Flag",
                       "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1",
                       "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2"
                       ], axis=1)
x = preproc_data_x.values
y = preproc_data["Attrition_Flag"].values
preproc_data_x

,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,-0.165261,1.060261,0.503636,-0.893897,0.522808,0.425471,-0.263546,0.384495,0.763915,-1.327615,0.492010,0.447062,-0.473285,0.489410,2.666758,-0.960392,-0.975049,3.892696,-0.776043
1,0.583481,1.060261,0.503636,0.593115,0.522808,1.308928,-0.263546,0.008956,0.120645,-1.327615,-2.220431,-0.573420,-1.426735,-0.445377,8.499347,-0.741457,-1.913197,6.910076,-0.997334
2,-0.789213,-0.943070,1.273662,-0.893897,0.522808,-0.850632,-0.263546,-0.241404,-0.522625,1.643518,-1.316284,-0.584975,1.661853,-0.733868,2.991048,-0.952436,-1.913197,6.910076,1.759732
3,-0.789213,1.060261,0.503636,-1.637403,0.522808,0.425471,-0.263546,-1.868743,0.763915,-1.327615,-2.220431,-0.430577,-1.426735,-0.302563,6.558239,-1.057042,-1.572052,7.621803,-0.997334
4,-0.290051,1.060261,-0.266389,0.593115,0.522808,-0.163500,-0.263546,0.008956,-0.522625,-1.327615,-0.412137,-0.508271,0.103448,-0.517445,2.856699,-0.976893,-1.742625,0.572725,0.130886
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10115,0.458691,1.060261,-0.266389,0.593115,-1.135236,-0.163500,-0.263546,0.509675,-0.522625,-0.337237,0.492010,-0.509042,0.844611,-0.584671,-0.261119,3.262728,2.223186,0.619605,0.678672
10116,-0.664422,1.060261,-0.266389,0.593115,2.180853,-0.163500,-0.263546,-1.368023,0.120645,-0.337237,0.492010,-0.478888,1.255686,-0.591383,0.206786,1.284945,0.176316,-0.121955,0.856430
10117,-0.290051,-0.943070,-1.036415,-0.893897,0.522808,-0.850632,-0.263546,0.008956,0.763915,0.653140,1.396157,-0.354313,-1.426735,-0.226314,0.276276,1.734897,-0.207472,0.453393,-0.997334
10118,-2.037116,1.060261,-0.266389,0.593115,0.522808,-0.163500,-0.263546,0.008956,0.120645,0.653140,0.492010,-0.368399,-1.426735,-0.240398,-1.039415,1.176215,-0.122186,0.044257,-0.997334


# data type identifying

In [6]:

categorical_features = ['Attrition_Flag', 'Gender', 'Marital_Status']
ordinal_features = ["Dependent_count", "Education_Level", "Card_Category",
                "Income_Category", "Total_Relationship_Count",
                "Months_Inactive_12_mon", "Contacts_Count_12_mon"]
numerical_features = ['Credit_Limit', 'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1',
                      'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio',
                      'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1',
                      'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2',
                      'Total_Trans_Ct', 'Total_Revolving_Bal', 'Total_Trans_Amt', 'Customer_Age', 'Months_on_book']

# Pair Correlation features selection

In [7]:
corr_manage_matrix = preproc_data.drop(["Attrition_Flag"], axis=1).corr() > 0.6
for idx in range(corr_manage_matrix.shape[0]):
    corr_manage_matrix.iloc[idx, idx] = False

In [8]:
drop_labels = [i[0] for i in filter(lambda x: x[1] == False, corr_manage_matrix.any().items())]
help_manage_matrix = corr_manage_matrix[corr_manage_matrix.any()].drop(drop_labels, axis=1)
high_correlation_val_list = []# consists tuples: (feature1, feature2), features(1-2) have high(> 0.6) correlation coefficient related to each other
forbidden_labels_list = []
for corr_feature_j in help_manage_matrix.columns:
    corr_feature_i = list(filter(lambda x: x[1] == True, help_manage_matrix[corr_feature_j].items()))[0][0]
    if corr_feature_i not in forbidden_labels_list:
        high_correlation_val_list.append([corr_feature_i, corr_feature_j])
    forbidden_labels_list.append(corr_feature_j)

high_correlation_val_list

[['Months_on_book', 'Customer_Age'],
 ['Income_Category', 'Gender'],
 ['Avg_Open_To_Buy', 'Credit_Limit'],
 ['Avg_Utilization_Ratio', 'Total_Revolving_Bal'],
 ['Total_Trans_Ct', 'Total_Trans_Amt']]

In [15]:
preproc_data_copy = preproc_data.copy()

first best feature recognizing method type
accuracy = 0.9221343873517787
time elapsed 11.697004556655884

In [9]:
trash_features = []
for corr_feature_couple in high_correlation_val_list:
    curr_corr_data = preproc_data.corr()[corr_feature_couple + ["Attrition_Flag"]]
    feature1_target_corr = curr_corr_data[corr_feature_couple[0]]["Attrition_Flag"]
    feature2_target_corr = curr_corr_data[corr_feature_couple[1]]["Attrition_Flag"]
    if feature1_target_corr > feature2_target_corr:
        trash_features.append(corr_feature_couple[1])
    else:
        trash_features.append(corr_feature_couple[0])

trash_features


['Months_on_book',
 'Gender',
 'Credit_Limit',
 'Total_Revolving_Bal',
 'Total_Trans_Ct']

second best feature recognizing method type
accuracy = 0.917786561264822
time elapsed 12.080619812011719

In [168]:
trash_features = []
for corr_feature_couple in high_correlation_val_list:
    feature1_2_metrics = []
    for drop_feature in corr_feature_couple:
        clf_model = RandomForestClassifier()
        score = cross_val_score(clf_model, X=preproc_data_x.drop(drop_feature, axis=1).values,
                                y=y,
                                cv=5, scoring="accuracy").mean()
        feature1_2_metrics.append(score)

    if feature1_2_metrics[0] > feature1_2_metrics[1]:
        trash_features.append(corr_feature_couple[1])
    else:
        trash_features.append(corr_feature_couple[0])

In [169]:
trash_features

['Customer_Age',
 'Income_Category',
 'Avg_Open_To_Buy',
 'Total_Revolving_Bal',
 'Total_Trans_Amt']

In [17]:
preproc_data_x

,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,...,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
0,1,-0.165261,1.060261,0.503636,-0.893897,0.522808,0.425471,-0.263546,0.384495,0.763915,...,0.447062,-0.473285,0.489410,2.666758,-0.960392,-0.975049,3.892696,-0.776043,-0.437752,0.437762
1,1,0.583481,1.060261,0.503636,0.593115,0.522808,1.308928,-0.263546,0.008956,0.120645,...,-0.573420,-1.426735,-0.445377,8.499347,-0.741457,-1.913197,6.910076,-0.997334,-0.437950,0.437953
2,1,-0.789213,-0.943070,1.273662,-0.893897,0.522808,-0.850632,-0.263546,-0.241404,-0.522625,...,-0.584975,1.661853,-0.733868,2.991048,-0.952436,-1.913197,6.910076,1.759732,-0.437642,0.437652
3,1,-0.789213,1.060261,0.503636,-1.637403,0.522808,0.425471,-0.263546,-1.868743,0.763915,...,-0.430577,-1.426735,-0.302563,6.558239,-1.057042,-1.572052,7.621803,-0.997334,-0.437949,0.437953
4,1,-0.290051,1.060261,-0.266389,0.593115,0.522808,-0.163500,-0.263546,0.008956,-0.522625,...,-0.508271,0.103448,-0.517445,2.856699,-0.976893,-1.742625,0.572725,0.130886,-0.437857,0.437844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10115,1,0.458691,1.060261,-0.266389,0.593115,-1.135236,-0.163500,-0.263546,0.509675,-0.522625,...,-0.509042,0.844611,-0.584671,-0.261119,3.262728,2.223186,0.619605,0.678672,-0.437484,0.437488
10116,0,-0.664422,1.060261,-0.266389,0.593115,2.180853,-0.163500,-0.263546,-1.368023,0.120645,...,-0.478888,1.255686,-0.591383,0.206786,1.284945,0.176316,-0.121955,0.856430,2.286428,-2.286431
10117,0,-0.290051,-0.943070,-1.036415,-0.893897,0.522808,-0.850632,-0.263546,0.008956,0.763915,...,-0.354313,-1.426735,-0.226314,0.276276,1.734897,-0.207472,0.453393,-0.997334,2.293573,-2.293577
10118,0,-2.037116,1.060261,-0.266389,0.593115,0.522808,-0.163500,-0.263546,0.008956,0.120645,...,-0.368399,-1.426735,-0.240398,-1.039415,1.176215,-0.122186,0.044257,-0.997334,2.290370,-2.290360


In [177]:
preproc_data_x = preproc_data_x.drop(trash_features, axis=1)

# End of Pair Correlation features selection


# Features selection by it's positive, negative target feature ration for each feature

In [22]:
trash_features = analyze_data(preproc_data.drop(numerical_features, axis=1), "Attrition_Flag", 5)


Gender pos/neg target ratio:
Gender : 1.0602613969424604 ,ratio = 4067 / 697 = 5.835 	pos_perc = 85.37%, neg_perc = 14.63%
Gender : -0.943070443434257 ,ratio = 4427 / 929 = 4.765 	pos_perc = 82.65%, neg_perc = 17.35%

Dependent_count pos/neg target ratio:
Dependent_count : 0.5036361663392939 ,ratio = 2248 / 481 = 4.674 	pos_perc = 82.37%, neg_perc = 17.63%
Dependent_count : 1.2736615483235292 ,ratio = 1313 / 260 = 5.05 	pos_perc = 83.47%, neg_perc = 16.53%
Dependent_count : -0.2663892156449413 ,ratio = 2238 / 417 = 5.367 	pos_perc = 84.29%, neg_perc = 15.71%
Dependent_count : -1.806439979613412 ,ratio = 768 / 135 = 5.689 	pos_perc = 85.05%, neg_perc = 14.95%
Dependent_count : 2.0436869303077643 ,ratio = 359 / 64 = 5.609 	pos_perc = 84.87%, neg_perc = 15.13%
Dependent_count : -1.0364145976291763 ,ratio = 1568 / 269 = 5.829 	pos_perc = 85.36%, neg_perc = 14.64%

Education_Level pos/neg target ratio:
Education_Level : -0.8938971384076957 ,ratio = 1706 / 306 = 5.575 	pos_perc = 84.79%, ne

['Gender',
 'Dependent_count',
 'Education_Level',
 'Marital_Status',
 'Income_Category',
 'Card_Category']

In [ ]:
trash_features

In [ ]:
preproc_data_x = preproc_data_x.drop(trash_features, axis=1)


# use inbuilt functions of python's classifiers in order to detect most useful and useless features for classification
# SelectKBest
accuracy = 0.942094861660079
time elapsed 14.244200944900513

In [10]:
from sklearn.feature_selection import SelectKBest, f_regression
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3,
                                                    random_state=1)
best_features_number = 9 # chose according next code cell from below
SelectKBest_model = SelectKBest(score_func=f_regression,
                                k=best_features_number)
k_best_features = SelectKBest_model.fit_transform(x, y)

In [11]:
# code cell
sorted(SelectKBest_model.fit(x, y).scores_)

[8.47949343818261e-05,
 0.13827982500625782,
 1.059008554620853,
 1.9522707578480856,
 2.2334342183443767,
 3.4988591966075893,
 3.6374326954185365,
 4.476194841444998,
 5.6139904704396555,
 13.792285257807805,
 177.2672936043848,
 232.61843614599215,
 242.7260895237001,
 295.82303148097924,
 336.087560516785,
 440.8360408901241,
 755.5460931774444,
 947.9056751516019,
 1622.469516245955]

In [12]:
scores = pd.concat([pd.DataFrame(preproc_data_x.columns),
                    pd.DataFrame(SelectKBest_model.fit(x, y).scores_)],
                   axis=1)
scores

,0,0
0,Customer_Age,3.498859
1,Gender,13.792285
2,Dependent_count,3.637433
3,Education_Level,4.476195
4,Marital_Status,2.233434
5,Income_Category,1.059009
6,Card_Category,0.138280
7,Months_on_book,1.952271
8,Total_Relationship_Count,232.618436
9,Months_Inactive_12_mon,242.726090


In [18]:
sorted(list(zip(scores.iloc[:,0], scores.iloc[:,1])), key=lambda x: x[1], reverse=True)

[('Total_Trans_Ct', 1622.469516245955),
 ('Total_Ct_Chng_Q4_Q1', 947.9056751516019),
 ('Total_Revolving_Bal', 755.5460931774444),
 ('Contacts_Count_12_mon', 440.8360408901241),
 ('Avg_Utilization_Ratio', 336.087560516785),
 ('Total_Trans_Amt', 295.82303148097924),
 ('Months_Inactive_12_mon', 242.7260895237001),
 ('Total_Relationship_Count', 232.61843614599215),
 ('Total_Amt_Chng_Q4_Q1', 177.2672936043848),
 ('Gender', 13.792285257807805),
 ('Credit_Limit', 5.6139904704396555),
 ('Education_Level', 4.476194841444998),
 ('Dependent_count', 3.6374326954185365),
 ('Customer_Age', 3.4988591966075893),
 ('Marital_Status', 2.2334342183443767),
 ('Months_on_book', 1.9522707578480856),
 ('Income_Category', 1.059008554620853),
 ('Card_Category', 0.13827982500625782),
 ('Avg_Open_To_Buy', 8.47949343818261e-05)]

In [13]:
k_best_features_mod = [k_best_features[:, i] for i in range(len(k_best_features[0]))]
def mod_lambda(x):
    for i in k_best_features_mod:
        if all(i == preproc_data_x[x].values):
            return True
    return False

k_best_features_name = list(filter(mod_lambda, preproc_data_x))

In [21]:
k_best_features_name

['Total_Relationship_Count',
 'Months_Inactive_12_mon',
 'Contacts_Count_12_mon',
 'Total_Revolving_Bal',
 'Total_Trans_Amt',
 'Total_Trans_Ct',
 'Total_Ct_Chng_Q4_Q1',
 'Avg_Utilization_Ratio']

In [22]:
trash_features = np.setdiff1d(np.array(preproc_data_x.columns), np.array(k_best_features_name))

In [23]:
trash_features

array(['Avg_Open_To_Buy', 'Card_Category', 'Credit_Limit', 'Customer_Age',
       'Dependent_count', 'Education_Level', 'Gender', 'Income_Category',
       'Marital_Status', 'Months_on_book', 'Total_Amt_Chng_Q4_Q1'],
      dtype=object)

In [190]:
preproc_data_x = preproc_data_x.drop(trash_features, axis=1)

In [161]:
preproc_data_x

,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Total_Revolving_Bal,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,0.763915,-1.327615,0.492010,-0.473285,2.666758,-0.960392,-0.975049,3.892696,-0.776043
1,0.120645,-1.327615,-2.220431,-1.426735,8.499347,-0.741457,-1.913197,6.910076,-0.997334
2,-0.522625,1.643518,-1.316284,1.661853,2.991048,-0.952436,-1.913197,6.910076,1.759732
3,0.763915,-1.327615,-2.220431,-1.426735,6.558239,-1.057042,-1.572052,7.621803,-0.997334
4,-0.522625,-1.327615,-0.412137,0.103448,2.856699,-0.976893,-1.742625,0.572725,0.130886
...,...,...,...,...,...,...,...,...,...
10115,-0.522625,-0.337237,0.492010,0.844611,-0.261119,3.262728,2.223186,0.619605,0.678672
10116,0.120645,-0.337237,0.492010,1.255686,0.206786,1.284945,0.176316,-0.121955,0.856430
10117,0.763915,0.653140,1.396157,-1.426735,0.276276,1.734897,-0.207472,0.453393,-0.997334
10118,0.120645,0.653140,0.492010,-1.426735,-1.039415,1.176215,-0.122186,0.044257,-0.997334


# feature selection using LogisticRegression
accuracy = 0.9390316205533598
time elapsed 11.121315002441406

In [194]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel

selector = SelectFromModel(LogisticRegression(penalty="l1", C=0.01,
                                              solver="liblinear"))
selector.fit(X=x, y=y)
manage_array = selector.get_support()

In [130]:
selector.estimator_.coef_

array([[ 0.05101883,  0.47492018, -0.17400038, -0.04965708,  0.1704445 ,
        -0.23288901, -0.11245795,  0.04651969,  0.69811067, -0.51250684,
        -0.55572608,  0.02613341,  0.77488566,  0.09119238,  0.09066267,
        -1.604506  ,  2.70611948,  0.65569399,  0.03027849]])

In [142]:
manage_array

array([False,  True,  True, False,  True, False, False, False,  True,
        True,  True, False,  True, False, False,  True,  True,  True,
       False])

TypeError: bad operand type for unary ~: 'Index'

In [195]:
preproc_data_x = preproc_data_x[preproc_data_x.columns[manage_array]]

# model evaluation(checking feature selection methods useness)
metrics without features selection
accuracy = 0.9468379446640316
time elapsed 16.462324619293213

In [196]:
x = preproc_data_x.values
y = preproc_data["Attrition_Flag"].values

In [197]:
import time

clf_model_best_params = {'bootstrap': False,
                         'max_depth': 12,
                         'max_features': 'sqrt',
                         'n_estimators': 100}
clf_model = RandomForestClassifier(**clf_model_best_params)

t0 = time.time()
print(cross_val_score(clf_model, x, y, cv=10, scoring="accuracy").mean())
print(f"time elapsed {time.time() - t0}")

0.9390316205533598
time elapsed 11.121315002441406


In [193]:

preproc_data = pd.read_csv("pre-processing_BankChurners.csv")
preproc_data_x = preproc_data.drop(["Attrition_Flag",
                       "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1",
                       "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2"
                       ], axis=1)
x = preproc_data_x.values
y = preproc_data["Attrition_Flag"].values

In [178]:
preproc_data_x


,Gender,Dependent_count,Education_Level,Marital_Status,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Total_Amt_Chng_Q4_Q1,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1
0,1.060261,0.503636,-0.893897,0.522808,-0.263546,0.384495,0.763915,-1.327615,0.492010,0.447062,-0.473285,2.666758,-0.975049,3.892696
1,1.060261,0.503636,0.593115,0.522808,-0.263546,0.008956,0.120645,-1.327615,-2.220431,-0.573420,-1.426735,8.499347,-1.913197,6.910076
2,-0.943070,1.273662,-0.893897,0.522808,-0.263546,-0.241404,-0.522625,1.643518,-1.316284,-0.584975,1.661853,2.991048,-1.913197,6.910076
3,1.060261,0.503636,-1.637403,0.522808,-0.263546,-1.868743,0.763915,-1.327615,-2.220431,-0.430577,-1.426735,6.558239,-1.572052,7.621803
4,1.060261,-0.266389,0.593115,0.522808,-0.263546,0.008956,-0.522625,-1.327615,-0.412137,-0.508271,0.103448,2.856699,-1.742625,0.572725
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10115,1.060261,-0.266389,0.593115,-1.135236,-0.263546,0.509675,-0.522625,-0.337237,0.492010,-0.509042,0.844611,-0.261119,2.223186,0.619605
10116,1.060261,-0.266389,0.593115,2.180853,-0.263546,-1.368023,0.120645,-0.337237,0.492010,-0.478888,1.255686,0.206786,0.176316,-0.121955
10117,-0.943070,-1.036415,-0.893897,0.522808,-0.263546,0.008956,0.763915,0.653140,1.396157,-0.354313,-1.426735,0.276276,-0.207472,0.453393
10118,1.060261,-0.266389,0.593115,0.522808,-0.263546,0.008956,0.120645,0.653140,0.492010,-0.368399,-1.426735,-1.039415,-0.122186,0.044257
